# 典范嵌入及其实现
## canonical embedding 典范嵌入
典范嵌入是一类泛指的嵌入，指的是 ***原环元素在扩张结构中的”自然对应“***
简单的典范嵌入有$Z->Q$以
我们处理的典范嵌入则是如下图所示：
$$
\sigma:C[X]/(X^{N}+1)=>C^{N}
$$
典范嵌入将分圆多项式$\Phi_{M}(X)=X^{N}+1$的各个根$\xi,\xi^{3},...,\xi^{2N-1}$带入目标多项式$C[X]/(X^{N}+1)$中逐个evaluate，然后得到的根组合成$C^{N}$
也即：
$$
\begin{aligned}
\sigma(m)&=(m(\xi),m(\xi^{3}),...,m(\xi^{2N-1}))\\
&= (z_{1},...,z_{N})
\end{aligned}
$$
注意，这里的根是从1到2N-1而不是N的

典范嵌入σ定义了一个同构（也就是说它定义了一个双射同态），在计算上它是同态的，在映射上是双射的

### 分圆多项式的根
$$
\phi_{n}(x)=\prod\limits_{1\le k\le n,gcd(k,n)=1}(x-e^{2i\pi \frac{k}{n}})
$$

当$N=2^{k}$时，有
$$
\phi_{2N}(X)=X^{N}+1
$$

### 双射的说明
已知：
$$m(X)=\sum\limits_{i=0}^{N-1}=\alpha_{i}X^{i} \in C[X]/(X^{N}+1)$$
评估是如下进行的：
$$\sum\limits_{j=0}^{N-1}\alpha(\xi^{2i-1})^{j}=z_{i},i=1,...,N$$

因此我们可以将其看作一个矩阵乘法：
$$A\alpha=z$$

由于A是一个范德蒙矩阵，且构成$x$的根各不相同，因此存在逆矩阵

In [6]:
# 以下是典范嵌入的代码实现
import numpy as np
from math import gcd

M = 8
assert(M & (M - 1) == 0), "M必须是2的幂次方"
N = M // 2 
vectors = np.array([1, 2, 2, 2])

# 构造典范嵌入相关的矩阵
roots = [np.exp(2j * np.pi * k / M) for k in range(M) if gcd(k, M) == 1] # 通过分园多项式结论得知：该式子得到X^N+1对应的所有单位根(M=2N)

A = np.array([[root ** i for i in range(N)] for root in roots])

# 应用典范嵌入逆运算，将向量映射回多项式系数
poly_coeffs = np.linalg.solve(A, vectors)
print("多项式系数:", poly_coeffs)

# 验证：将多项式系数通过典范嵌入映射回向量
reconstructed_vectors = A @ poly_coeffs
print("重构向量:", reconstructed_vectors)


多项式系数: [ 1.75000000e+00+7.96951489e-17j -1.76776695e-01+1.76776695e-01j
 -7.63278329e-17+2.50000000e-01j  1.76776695e-01+1.76776695e-01j]
重构向量: [1.-8.12941988e-18j 2.+5.61862510e-17j 2.+6.70078871e-17j
 2.+1.03204047e-16j]


In [ ]:
# 调用encoding模块中的函数进行编码和解码
from encoding import SimpleEncoding
import numpy as np

M = 8
# 测试加法和乘法同态
encoder = SimpleEncoding(M)
v1 = np.array([1, 2, 3, 4])
v2 = np.array([5, 6, 7, 8])

modulo = np.polynomial.Polynomial([1] + [0]*(M//2 - 1) + [1])  # X^N + 1

ct1 = encoder.sigma_inv(v1)
ct2 = encoder.sigma_inv(v2)

ct_add = ct1 + ct2
# 考虑到多项式相乘，最大项会超出范围，因此我们需要对其进行模X^N+1的约简
ct_mul = (ct1 * ct2) % modulo

decoded_add = encoder.sigma(ct_add)
decoded_mul = encoder.sigma(ct_mul)

print(f"Added ciphertexts decoded: {decoded_add} (expected: {v1 + v2})") # 实际上并不是密文，只是这么叫而已
print(f"Multiplied ciphertexts decoded: {decoded_mul} (expected: {v1 * v2})")

print(f"Add norm: {np.linalg.norm(decoded_add - (v1 + v2))}")
print(f"Mul norm: {np.linalg.norm(decoded_mul - (v1 * v2))}")

Added ciphertexts decoded: [ 6.+3.45793845e-17j  8.-2.18527158e-16j 10.+3.53998674e-16j
 12.-8.59671258e-17j] (expected: [ 6  8 10 12])
Multiplied ciphertexts decoded: [ 5.+6.34808976e-16j 12.-6.12064760e-16j 21.+2.39285973e-15j
 32.+1.27112551e-15j] (expected: [ 5 12 21 32])
Add norm: 4.26210347928688e-16
Mul norm: 3.3577651883537623e-15


# $Z[X]/(X^N+1)$上的典范嵌入

![[Pasted image 20251111170719.png]]
由上图的N=4的简单情况可知，分圆多项式的根实际上是对称的。
在这个例子中，有$\omega_{1}=\overline{\omega_{7}}$，$\omega_{3}=\overline{\omega_{5}}$
考虑到总数=8，我们就有了$\omega_{j}=\overline{\omega_{-j}}$

由于在$m(x) \in Z[X]$中做评估，因此就有了$m(\xi^{j})=\overline{m(\xi^{-j})}=m(\overline{\xi^{-j}})$

由于$\sigma$映射中的每一个向量元素都是由多项式在单位根上评估而来，因此我们有：
$$
\begin{align}
Z_{N}&= (z_{1},...,z_{N})\\
     &= (m(\xi),m(\xi^{3}),...,m(\xi^{2N-1})) \\
     &= (m(\xi),m(\xi^{3}),...,m(\overline{\xi^{3}}),m(\overline{\xi})) \\
     &= (z_{1},z_{2},...,\overline{z_{2}},\overline{z_{1}})
\end{align}
$$

因此，需要在实数参数的$m(x)$的情况下，评估出来的$Z_{N}$实际上自由度只有$N/2$

从典范嵌入的正方向的例子可以说明（但是不是证明），如果我们想要保证典范嵌入的逆方向$\sigma^{-1}:C^{N} \to Z[X]/(X^N+1)$，复向量映射到$Z[X]/(X^N+1)$，我们至少要保证$C^{N}$的自由度减半，也就是变为$C^{N/2}$

从以下的代码以及输出也可以看出相关的结论：我们会发现，当M=8，N=4是，如果输入vector是形如上文的$Z_{N}$的形式，那么转换出来的多项式是实数的，如果输入的vector不是这样，那么转换的多项式$m(x)\notin Z[X]$

In [31]:
from encoding import SimpleEncoding

M = 8

encoder = SimpleEncoding(M)

v1 = np.array([1, 2, 2, 1])
v2 = np.array([1, 2, 2, 2])
v3 = np.array([1+1j, 2+2j, 2-2j, 1-1j])
v4 = np.array([1+1j, 2+2j, 2+2j, 1-1j])
v5 = np.array([1j, 2j, -2j, -1j])
v6 = np.array([1j, 2j, 2j, -1j])

ct1 = encoder.sigma_inv(v1)
ct2 = encoder.sigma_inv(v2)
ct3 = encoder.sigma_inv(v3)
ct4 = encoder.sigma_inv(v4)
ct5 = encoder.sigma_inv(v5)
ct6 = encoder.sigma_inv(v6)

def is_effectively_real(x, tol=1e-15):
    x = np.asarray(x)
    return np.all(np.abs(np.imag(x)) < tol)

print(f"origin: {v1} sigma^{{-1}} is real: {is_effectively_real(ct1.coef)}")
print(f"origin: {v2} sigma^{{-1}} is real: {is_effectively_real(ct2.coef)}")
print(f"origin: {v3} sigma^{{-1}} is real: {is_effectively_real(ct3.coef)}")
print(f"origin: {v4} sigma^{{-1}} is real: {is_effectively_real(ct4.coef)}")
print(f"origin: {v5} sigma^{{-1}} is real: {is_effectively_real(ct5.coef)}")
print(f"origin: {v6} sigma^{{-1}} is real: {is_effectively_real(ct6.coef)}")

origin: [1 2 2 1] sigma^{-1} is real: True
origin: [1 2 2 2] sigma^{-1} is real: False
origin: [1.+1.j 2.+2.j 2.-2.j 1.-1.j] sigma^{-1} is real: True
origin: [1.+1.j 2.+2.j 2.+2.j 1.-1.j] sigma^{-1} is real: False
origin: [ 0.+1.j  0.+2.j -0.-2.j -0.-1.j] sigma^{-1} is real: True
origin: [ 0.+1.j  0.+2.j  0.+2.j -0.-1.j] sigma^{-1} is real: False
